# Frequent Itemset Mining

Learning Objectives:

- Extract frequent patterns given a corpus of data.
- Find the rules which are interesting and non-obvious for a given domain.

In [1]:
# !pip install --user mlxtend

In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df1 = pd.read_excel('online_retail_II/Online Retail.xlsx')
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df2 = pd.read_excel('online_retail_II/online_retail_II.xlsx')
df2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Question 1.1: Concatinate both dataframes to create a single dataframe. Remove any rows where InvoiceNo is Null and Quantity is Negative

In [4]:
# df1.info()
# df2.info()
df2.rename(columns={"Invoice": "InvoiceNo", "Price": "UnitPrice", "Customer ID": "CustomerID"}, inplace=True)

In [5]:
frames = [df1, df2]
data = pd.concat(frames)
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
res = data.isnull().sum()
res[res > 0]
# None of the invoices are null

Description      4382
CustomerID     243007
dtype: int64

In [7]:
print("Total dataframe size:", len(data))
data.drop(data[data['Quantity'] < 0].index, inplace=True)
print("Size of dataframe after removing negative quantity rows:", len(data))

Total dataframe size: 1067370
Size of dataframe after removing negative quantity rows: 1022104


Question 1.2: Filter the data by only transactions that happened in United Kingdom 

In [8]:
data_uk = data.loc[(data['Country'] == 'United Kingdom'), :]
data_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Question 1.3: What are the most popular 5 items?

In [9]:
group_trans = data_uk.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False)

In [10]:
group_trans.head(5)

StockCode
84077     98713
85123A    90953
85099B    86144
22197     82990
23843     80995
Name: Quantity, dtype: int64

Question 1.4: Filter down the data to include transaction that contain the top 20 items

In [11]:
group_trans.head(20)

StockCode
84077     98713
85123A    90953
85099B    86144
22197     82990
23843     80995
23166     77000
84879     74556
21212     70783
17003     70003
21977     45702
84991     40835
15036     39509
22178     38613
22616     37644
84946     37006
84755     36993
22086     34886
21232     34784
22386     34649
85099F    34297
Name: Quantity, dtype: int64

Question 2.1: Consolidate the items into 1 transaction per row and each product one-hot encoded.

In [21]:
#Create the "basket"
basket = data_uk.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')

In [22]:
basket.head()
# len(data_uk['StockCode'].unique())

Description,22719,DOORMAT UNION JACK GUNS AND ROSES,3 STRIPEY MICE FELTCRAFT,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,ANIMAL STICKERS,BLACK PIRATE TREASURE CHEST,BROWN PIRATE TREASURE CHEST,Bank Charges,CAMPHOR WOOD PORTOBELLO MUSHROOM,...,tk maxx mix up with pink,to push order througha s stock was,update,website fixed,wrong invc,wrongly coded 20713,wrongly coded 23343,wrongly marked,wrongly marked 23343,wrongly sold (22719) barcode
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
489434,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Question 2.2: Convert all the values to 1 when values are greater than 0 and 0 when values are 0 or less.

In [23]:
def encode(x):
    if x <= 0:
        return 0
    if x > 0:
        return 1

basket_sets = basket.applymap(encode)
basket_sets.head()

Description,22719,DOORMAT UNION JACK GUNS AND ROSES,3 STRIPEY MICE FELTCRAFT,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,ANIMAL STICKERS,BLACK PIRATE TREASURE CHEST,BROWN PIRATE TREASURE CHEST,Bank Charges,CAMPHOR WOOD PORTOBELLO MUSHROOM,...,tk maxx mix up with pink,to push order througha s stock was,update,website fixed,wrong invc,wrongly coded 20713,wrongly coded 23343,wrongly marked,wrongly marked 23343,wrongly sold (22719) barcode
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
489434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Question 3.1: Apply [apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) algorithm to generate frequent item sets that have a support of at least 7%

In [25]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
frequent_itemsets

/Users/standard/.local/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.051916,(60 TEATIME FAIRY CAKE CASES)
1,0.070944,(ASSORTED COLOUR BIRD ORNAMENT)
2,0.051916,(HEART OF WICKER LARGE)
3,0.056099,(HEART OF WICKER SMALL)
4,0.053639,(HOME BUILDING BLOCK WORD)
5,0.081606,(JUMBO BAG RED RETROSPOT)
6,0.056455,(JUMBO SHOPPER VINTAGE RED PAISLEY)
7,0.059489,(JUMBO STORAGE BAG SUKI)
8,0.060364,(LUNCH BAG BLACK SKULL.)
9,0.050905,(LUNCH BAG CARS BLUE)


Question 3.2: Generate the association rules with their corresponding support, confidence and lift.

In [26]:
association_rules(frequent_itemsets, metric='lift')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


Question 4: Based on the above rules, identify what would be the opportunity of promoting one of the antecendents.

I don't see any rules generated for 5% support also. I double checked the concatenation of the 2 data frames (seems to be right) and I'm grouping by StockCode. Even when I group by description when creating basket with both 5% and 7% support, I'm not seeing association rules. Not sure what I'm doing wrong, I'm hoping to go over the answer key that Mohamed will provide.

Question 5: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

Incoming experience: No incoming experience Steps taken: This week's lesson was about frequent itemset mining. Implemented Apriori algorithm APIs Obstacles: This week's obstacle was getting the association_rules API to generate association rules Link to real world: Helped me understand how the APIs work and how the algorithm works with real world data Steps missing (with just this week's learning): Might be useful to implement and compare Holt's seasonal exponential smoothing and SARIMA